<a href="https://colab.research.google.com/github/Shami4004/vqa_dataset_project/blob/main/OCR_Images_Cleaned_Arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jun 20 03:48:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# STEP 1: Install dependencies
!pip install easyocr --quiet
!pip install arabic_reshaper python-bidi --quiet
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import zipfile

zip_path = "/content/drive/MyDrive/arabic_images.zip"  # path in your Drive
extract_dir = "/content/arabic_images"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Dataset extracted to:", extract_dir)

✅ Dataset extracted to: /content/arabic_images


In [5]:
import easyocr
from PIL import Image
import shutil

reader = easyocr.Reader(['ar'])  # Arabic only

SOURCE_DIR = "/content/arabic_images"
DEST_DIR = "/content/arabic_images_cleaned"
os.makedirs(DEST_DIR, exist_ok=True)

def contains_arabic(text):
    return any('\u0600' <= c <= '\u06FF' for c in text)

def has_watermark(text, image_size):
    return len(text) >= 3 and image_size[0] < 700 and image_size[1] < 700  # crude filter

count = 0
total = 0

for root, _, files in os.walk(SOURCE_DIR):
    for fname in files:
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img_path = os.path.join(root, fname)
        try:
            img = Image.open(img_path).convert("RGB")
            result = reader.readtext(img_path, detail=0)

            total += 1
            arabic_texts = [t for t in result if contains_arabic(t)]

            if arabic_texts and not has_watermark(" ".join(result), img.size):
                shutil.copy(img_path, os.path.join(DEST_DIR, fname))
                count += 1

            if total % 500 == 0:
                print(f"Processed {total} images... Arabic-only: {count}")

        except Exception as e:
            print(f"❌ Error on {fname}: {e}")
            continue

print(f"\n✅ Done! Arabic-only images kept: {count} / {total}")

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Processed 500 images... Arabic-only: 306


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Processed 37000 images... Arabic-only: 18775


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in by

Processed 37500 images... Arabic-only: 19104
Processed 38000 images... Arabic-only: 19305


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



✅ Done! Arabic-only images kept: 19410 / 38250


In [6]:
import shutil

zip_name = "/content/drive/MyDrive/arabic_images_cleaned.zip"
shutil.make_archive("/content/arabic_images_cleaned", 'zip', DEST_DIR)
shutil.move("/content/arabic_images_cleaned.zip", zip_name)

print(f"✅ Zipped and uploaded to Google Drive: {zip_name}")

✅ Zipped and uploaded to Google Drive: /content/drive/MyDrive/arabic_images_cleaned.zip


In [7]:
# ✅ Install Dependencies
!pip install easyocr
!pip install langdetect
!apt install zip -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=f81e7657026b83b9888af91e12293a2f62082b6228696562d9bb1af44f0ed5c0
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [8]:
# ✅ IMPORTS
import os
import shutil
from langdetect import detect
import easyocr
from PIL import Image, UnidentifiedImageError
from tqdm import tqdm

In [9]:
# ✅ CONFIG
SOURCE_DIR = "/content/arabic_images"  # Upload your 38k dataset here
DEST_DIR = "/content/arabic_images_cleaned"
os.makedirs(DEST_DIR, exist_ok=True)

# Forbidden terms (watermark checks)
WATERMARK_TERMS = ["shutterstock", "alamy", "123rf", "dreamstime", "getty", "istock", "depositphotos"]

In [10]:
# ✅ Init OCR
reader = easyocr.Reader(['ar'], gpu=True)  # Use GPU for speed if available

In [11]:
# ✅ FILTERING FUNCTION
def is_valid_arabic(text):
    try:
        lang = detect(text)
        if lang != 'ar':
            return False
        for term in WATERMARK_TERMS:
            if term.lower() in text.lower():
                return False
        return True
    except:
        return False

In [12]:
# ✅ MAIN FILTERING LOOP
total = 0
kept = 0

for root, _, files in tqdm(os.walk(SOURCE_DIR), desc="🔍 Filtering"):
    for fname in files:
        if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        total += 1
        img_path = os.path.join(root, fname)

        try:
            # Validate image
            with Image.open(img_path) as img:
                img.verify()

            result = reader.readtext(img_path, detail=0, paragraph=True)
            text = " ".join(result).strip()

            if text and is_valid_arabic(text):
                # ✅ Unique filename based on subfolder
                folder_name = os.path.basename(root)
                new_fname = f"{folder_name}_{fname}"
                shutil.copy(img_path, os.path.join(DEST_DIR, new_fname))
                kept += 1

        except UnidentifiedImageError:
            continue
        except Exception as e:
            continue

print(f"\n✅ Done! Arabic-only images kept: {kept} / {total}")

🔍 Filtering: 5it [01:19, 15.87s/it]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
🔍 Filtering: 9it [10:50, 75.00s/it]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
🔍 Filtering: 11it [12:52, 68.96s/it]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib


✅ Done! Arabic-only images kept: 18543 / 38255


In [13]:
# ✅ VERIFY NUMBER OF FILES
cleaned_files = os.listdir(DEST_DIR)
print(f"📸 Total final cleaned images: {len(cleaned_files)}")

📸 Total final cleaned images: 19328


In [14]:
import zipfile
from tqdm import tqdm

zip_path = "/content/arabic_images_cleaned.zip"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in tqdm(os.listdir(DEST_DIR), desc="📦 Zipping to local zip"):
        zipf.write(os.path.join(DEST_DIR, file), arcname=file)

print(f"✅ Zip file ready: {zip_path}")

📦 Zipping to local zip: 100%|██████████| 19328/19328 [04:19<00:00, 74.41it/s]

✅ Zip file ready: /content/arabic_images_cleaned.zip


In [15]:
from google.colab import files
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!git clone https://github.com/Shami4004/vqa_dataset_project.git
%cd vqa_dataset_project

Cloning into 'vqa_dataset_project'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 81 (delta 23), reused 69 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 734.77 KiB | 16.70 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/vqa_dataset_project
